In [4]:
# --- Install required libraries (Uncomment if running in Colab) ---
!pip install rake-nltk gensim nltk

import nltk
nltk.download('stopwords')

from rake_nltk import Rake
from nltk.corpus import stopwords
import string
import gensim
from gensim import corpora
from gensim.models import LdaModel
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/farisrizky/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/farisrizky/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [16]:


# -----------------------
# Sample Survey Responses
# -----------------------
responses = [
    "Pelayanan puskesmas sudah cukup baik, namun antrian terlalu lama.",
    "Saya berharap ada lebih banyak dokter dan jam buka diperpanjang.",
    "Obat sering tidak tersedia, terutama untuk pasien penyakit kronis.",
    "Petugas ramah dan sigap, tapi ruang tunggu sempit dan panas.",
    "Sebaiknya ada sistem antrean online agar lebih efisien.",
    "Petugasnya lucu, walaupun antrian membludak dan lama sekali."
]

# -----------------------
# RAKE Keyword Extraction
# -----------------------
print("🔹 RAKE Keyword Extraction:\n")

r = Rake(language='indonesian')  # Uses NLTK stopwords
for i, resp in enumerate(responses):
    r.extract_keywords_from_text(resp)
    print(f"Response {i+1}:")
    for score, phrase in r.get_ranked_phrases_with_scores():
        print(f"  - {phrase} ({score})")
    print()

# -----------------------
# Preprocessing for LDA
# -----------------------
print("\n🔹 Preprocessing for LDA...\n")

stop_words = set(stopwords.words('indonesian'))

def preprocess(doc):
    tokens = gensim.utils.simple_preprocess(doc, deacc=True)
    return [t for t in tokens if t not in stop_words and t not in string.punctuation]

processed_docs = [preprocess(doc) for doc in responses]

# -----------------------
# LDA Topic Modeling
# -----------------------
print("🔹 LDA Topic Modeling:\n")

# Create dictionary and corpus
dictionary = corpora.Dictionary(processed_docs)
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# Fit LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=2, passes=10, random_state=42)

# Show topics
topics = lda_model.print_topics(num_words=5)
for idx, topic in topics:
    print(f"Topic {idx+1}: {topic}")

# -----------------------
# Document Topic Assignment
# -----------------------
print("\n🔹 Document Topic Distribution:\n")

for i, row in enumerate(lda_model[corpus]):
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
    print(f"Response {i+1}: Dominant Topic {row[0][0]+1} (Score: {row[0][1]:.2f})")


🔹 RAKE Keyword Extraction:

Response 1:
  - pelayanan puskesmas (4.0)
  - antrian (1.0)

Response 2:
  - jam buka diperpanjang (9.0)
  - dokter (1.0)
  - berharap (1.0)

Response 3:
  - pasien penyakit kronis (9.0)
  - tersedia (1.0)
  - obat (1.0)

Response 4:
  - ruang tunggu sempit (9.0)
  - petugas ramah (4.0)
  - sigap (1.0)
  - panas (1.0)

Response 5:
  - sistem antrean online (9.0)
  - efisien (1.0)

Response 6:
  - petugasnya lucu (4.0)
  - antrian membludak (4.0)


🔹 Preprocessing for LDA...

🔹 LDA Topic Modeling:

Topic 1: 0.082*"antrian" + 0.049*"buka" + 0.049*"diperpanjang" + 0.049*"jam" + 0.049*"berharap"
Topic 2: 0.061*"petugas" + 0.061*"sigap" + 0.061*"tunggu" + 0.061*"panas" + 0.061*"sempit"

🔹 Document Topic Distribution:

Response 1: Dominant Topic 1 (Score: 0.87)
Response 2: Dominant Topic 1 (Score: 0.91)
Response 3: Dominant Topic 1 (Score: 0.91)
Response 4: Dominant Topic 2 (Score: 0.94)
Response 5: Dominant Topic 2 (Score: 0.90)
Response 6: Dominant Topic 1 (Scor

In [18]:
# --- Install required libraries if needed ---
!pip install sastrawi pyLDAvis

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import gensim
from gensim import corpora
from gensim.models import LdaModel

import string

# Optional visualization
import pyLDAvis.gensim_models
import pyLDAvis

# For display in Jupyter
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import matplotlib.pyplot as plt

# --------------------------
# Sastrawi Preprocessing
# --------------------------

# Initialize tools
stop_factory = StopWordRemoverFactory()
stop_remover = stop_factory.create_stop_word_remover()

stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

def preprocess_sastrawi(doc):
    cleaned = stop_remover.remove(doc.lower())
    stemmed = stemmer.stem(cleaned)
    tokens = gensim.utils.simple_preprocess(stemmed, deacc=True)
    return tokens

# Apply preprocessing
processed_docs = [preprocess_sastrawi(doc) for doc in responses]

print("🔹 Sample Processed Output:")
for i, tokens in enumerate(processed_docs):
    print(f"Response {i+1}: {tokens}")

# --------------------------
# LDA Topic Modeling
# --------------------------

# Create dictionary and corpus
dictionary = corpora.Dictionary(processed_docs)
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# Fit LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=2, passes=20, random_state=42)

# Show topics
print("\n🔹 LDA Topics (Top 5 words):")
topics = lda_model.print_topics(num_words=5)
for idx, topic in topics:
    print(f"Topic {idx+1}: {topic}")

# Document topic distribution
print("\n🔹 Dominant Topic Per Document:")
for i, row in enumerate(lda_model[corpus]):
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
    print(f"Response {i+1}: Dominant Topic {row[0][0]+1} (Score: {row[0][1]:.2f})")

# --------------------------
# pyLDAvis Visualization
# --------------------------

print("\n🔹 Launching pyLDAvis...")

pyLDAvis.enable_notebook()
vis_data = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis_data


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 2.9 MB/s eta 0:00:00a 0:00:01


Matplotlib is building the font cache; this may take a moment.


🔹 Sample Processed Output:
Response 1: ['layan', 'puskesmas', 'cukup', 'baik', 'antri', 'terlalu', 'lama']
Response 2: ['harap', 'lebih', 'banyak', 'dokter', 'jam', 'buka', 'panjang']
Response 3: ['obat', 'sering', 'sedia', 'utama', 'pasien', 'sakit', 'kronis']
Response 4: ['tugas', 'ramah', 'sigap', 'ruang', 'tunggu', 'sempit', 'panas']
Response 5: ['baik', 'sistem', 'antre', 'online', 'lebih', 'efisien']
Response 6: ['tugas', 'lucu', 'walaupun', 'antri', 'membludak', 'lama', 'sekali']

🔹 LDA Topics (Top 5 words):
Topic 1: 0.066*"antri" + 0.066*"lama" + 0.066*"baik" + 0.040*"tugas" + 0.039*"lebih"
Topic 2: 0.039*"lebih" + 0.038*"tugas" + 0.038*"sering" + 0.038*"pasien" + 0.038*"sakit"

🔹 Dominant Topic Per Document:
Response 1: Dominant Topic 1 (Score: 0.94)
Response 2: Dominant Topic 2 (Score: 0.93)
Response 3: Dominant Topic 2 (Score: 0.93)
Response 4: Dominant Topic 2 (Score: 0.93)
Response 5: Dominant Topic 1 (Score: 0.92)
Response 6: Dominant Topic 1 (Score: 0.93)

🔹 Launching py

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.065897  0.0       1        1  51.166359
0     -0.065897  0.0       2        1  48.833641, topic_info=       Term      Freq     Total Category  logprob  loglift
0     antri  1.000000  1.000000  Default  30.0000  30.0000
3      lama  1.000000  1.000000  Default  29.0000  29.0000
1      baik  1.000000  1.000000  Default  28.0000  28.0000
4     layan  1.000000  1.000000  Default  27.0000  27.0000
2     cukup  1.000000  1.000000  Default  26.0000  26.0000
..      ...       ...       ...      ...      ...      ...
8      buka  0.265284  1.070465   Topic2  -4.3238  -0.6783
13  panjang  0.265275  1.070465   Topic2  -4.3238  -0.6783
10    harap  0.265269  1.070465   Topic2  -4.3238  -0.6784
9    dokter  0.265248  1.070466   Topic2  -4.3239  -0.6784
20    utama  0.265079  1.070469   Topic2  -4.3245  -0.6791

[93 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
28        2  0.944180      antre
0         2  0.630599      antri
1         2  0.630597       baik
7         1  0.934174     banyak
8         1  0.934174       buka
2         2  0.944189      cukup
9         1  0.934173     dokter
29        2  0.944180    efisien
10        1  0.934173      harap
11        1  0.934174        jam
14        1  0.934169     kronis
3         2  0.630598       lama
4         2  0.944189      layan
12        1  0.626090      lebih
12        2  0.626090      lebih
32        2  0.944185       lucu
33        2  0.944185  membludak
15        1  0.934169       obat
30        2  0.944180     online
21        1  0.934174      panas
13        1  0.934173    panjang
16        1  0.934169     pasien
5         2  0.944189  puskesmas
22        1  0.934174      ramah
23        1  0.934175      ruang
17        1  0.934169      sakit
18        1  0.934169      sedia
34        2  0.944185     sekali
24        1  0.934175     sempit
19        1  0.934169     sering
25        1  0.934175      sigap
31        2  0.944180     sistem
6         2  0.944189    terlalu
26        1  0.626112      tugas
26        2  0.626112      tugas
27        1  0.934174     tunggu
20        1  0.934170      utama
35        2  0.944185   walaupun, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])